In [2]:
#architecture du code pour déplacer le capteur jusqu'à se trouver au niveau du rayon transmis ou réfléchi
#tous les angles donnés ici sont les angles du problème optique. Ils ne correspondent pas nécessairement aux angles
#moteurs dans le montage. A traiter une fois le montage fait.

import os, time,dynamixel, random
import sys
import numpy as np
from serial import Serial

def trouverPositionTransmis():
    
    #mettre le capteur en max(0,thetaS-10°)
    
    thetaS = moteur.getangle(0)
    thetamin = max(90,180-thetaS-10)
    
    #calculer la position limite, soit min(90,thetaS+10°)
    thetamax = min(180,180-thetaS+10)
    
    #initialiser les valeurs
    
    Imax = 0
    thetaC = 0
    posMin = #thetamin converti en intervalle moteur pour le moteur capteur
    posMax = #idem avec thetamax
    erreurs = []
    erreursBis = []
    
    #On balaye toutes les positions de l'intervalle en cherchant la position d'intensité maximale
    
    #pour le moment ce programme ne tient pas compte des erreurs éventuelles du moteur
    
    for i in range (0,4):
        k=0
        while posMin + i + k < posMax :
            if(moteur.move_motor(1,posMin+i+k)!=0):
                erreurs.append(posMin+i+k)
            if np.max(spec.intensities()) > Imax:
                Imax = np.max(spec.intensities())
                thetaC = moteur.getangle(1)
            k = k+5
    
    #On refait un passage sur les positions où on a eu une erreur (Est-ce nécessaire ? Est-ce suffisant ?)
    if len(erreurs)>0:
        for i in range(0,len(erreurs)):
            if(moteur.move_motor(1,erreurs[i]))!=0
                erreursBis.append(erreurs[i])
            if np.max(spec.intensities()) > Imax:
                Imax = np.max(spec.intensities())
                thetaC = moteur.getangle(1)
                
    #On refait un deuxième passage de sécurité
    if len(erreursBis)>0:
        compteur = 0
        for i in range(0,len(erreursBis))
            if moteur.move_motor(1,erreursBis[i])!=0:
                compteur = compteur + 1
            if np.max(spec.intensities()) > Imax:
                Imax = np.max(spec.intensities())
                thetaC = moteur.getangle(1)
    
    if compteur > 4:
        print("Un nombre important (", compteur, ") de positions n'ont pas pu être testée pour trouver la position du rayon transmis en theta=", thetaS,". Il est possible que cela engendre une erreur sur la mesure.")
    return thetaC

SyntaxError: invalid syntax (<ipython-input-2-b4a79aaa40c4>, line 24)

In [ ]:
#tous les angles donnés ici sont les angles du problème optique. Ils ne correspondent pas nécessairement aux angles
#moteurs dans le montage. A traiter une fois le montage fait.

import os, time,dynamixel, random
import sys
import numpy as np
from serial import Serial

def trouverPositionReflechi():
    
    #mettre le capteur en max(0,thetaS-10°)
    
    thetaS = moteur.getangle(0)
    thetamin = max(0,thetaS-10)
    
    #calculer la position limite, soit min(90,thetaS+10°)
    thetamax = min(90,thetaS+10)
    
    #initialiser les valeurs
    
    Imax = 0
    thetaC = 0
    posMin = #thetamin converti en intervalle moteur pour le moteur capteur
    posMax = #idem avec thetamax
    erreurs = []
    erreursBis = []
    
    #On balaye toutes les positions de l'intervalle en cherchant la position d'intensité maximale
    
    #pour le moment ce programme ne tient pas compte des erreurs éventuelles du moteur
    
    for i in range (0,4):
        k=0
        while posMin + i + k < posMax :
            if(moteur.move_motor(1,posMin+i+k)!=0):
                erreurs.append(posMin+i+k)
            if np.max(spec.intensities()) > Imax:
                Imax = np.max(spec.intensities())
                thetaC = moteur.getangle(1)
            k = k+5
    
    #On refait un passage sur les positions où on a eu une erreur (Est-ce nécessaire ? Est-ce suffisant ?)
    if len(erreurs)>0:
        for i in range(0,len(erreurs)):
            if(moteur.move_motor(1,erreurs[i]))!=0
                erreursBis.append(erreurs[i])
            if np.max(spec.intensities()) > Imax:
                Imax = np.max(spec.intensities())
                thetaC = moteur.getangle(1)
                
    #On refait un deuxième passage de sécurité
    if len(erreursBis)>0:
        compteur = 0
        for i in range(0,len(erreursBis))
            if moteur.move_motor(1,erreursBis[i])!=0:
                compteur = compteur + 1
            if np.max(spec.intensities()) > Imax:
                Imax = np.max(spec.intensities())
                thetaC = moteur.getangle(1)
    
    if compteur > 4:
        print("Un nombre important (", compteur, ") de positions n'ont pas pu être testée pour trouver la position du rayon réfléchi en theta=", thetaS,". Il est possible que cela engendre une erreur sur la mesure.")
    return thetaC

In [1]:
#Traitement de données

import seabreeze.spectrometers as sb
import time
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
%matplotlib notebook

def Integration_time(pas=1,T_0=10):#T_0 minimal vaux 10
    T=T_0
    pasSat = True
    while pasSat:
        spec = sb.Spectrometer.from_serial_number()
        spec.integration_time_micros(T)
        I_max=np.max(spec.intensities())
        spec.close()
        if  I_max > 62000:
            pasSat = False
        else:
            T+=pas
        time.sleep(0.01)
    T-=pas
    
    return T 

def I_mean(N,T):
    I=[0 for k in range(N)]
    spec = sb.Spectrometer.from_serial_number()
    Lambdas=spec.wavelengths()
    spec.close()
    for k in range(N):
        spec = sb.Spectrometer.from_serial_number()
        spec.integration_time_micros(T)
        I[k]=spec.intensities()
        spec.close()
        #plt.plot(Lambdas,I[k])
        time.sleep(0.01)
    Imean=np.mean(I,axis=0)
    #plt.plot(Lambdas,Imean,color='k',linewidth=2)
    return Lambdas,Imean 

def Interpolation(Lambdas,Imean,n=25):
    return Lambdas[int(n/2):len(Lambdas)-(int(n/2))],moving_average(Imean,n)
    
def moving_average(I, n, Lambdas) :
    I_averaged = np.cumsum(I, dtype=float)#Return the cumulative sum of A 
    I_averaged[n:] = I_averaged[n:] - I_averaged[:-n]
    Lambdas_same_size = Lambdas[int(n/2):len(Lambdas)-(int(n/2))]
    I_averaged = I_averaged[n - 1:] / n
    return Lambdas_same_size,I_averaged

def I_0(LAMBDAS,N=100,pas=1,T_0=10,n=25):
    '''return an array with the intensities of the light of the source between lambda_i and lambda_f every 5nm if step=5nm'''
    T=Integration_time(pas,T_0)
    Lambdas_tot,Imean=I_mean(N,T)
    Lambdas_tot,I_averaged = moving_average(Imean, n, Lambdas_tot)
    I_function=interp1d(Lambdas_tot,I_averaged)#Linear interpolation is the default one here
    return LAMBDAS,I_function(LAMBDAS)

In [1]:
#effectuer l'ensemble des mesures
import time

def effectuerMesures():
    
    #tous les angles utilisés sont les angles du problème OPTIQUE. Il faut les transformer en angles moteurs
    #selon la façon dont le montage réel sera réalisé
    #Programme SANS POLARISATION
    #On suppose que les moteurs ont déjà été initialisés
    
    acquisition = []
    
    for i in range (0,17): #pour chaque angle d'incidence possible, donc angle de la source (thetaS)
        
        #On commence par positionner la source
        thetaS = 2.5 + i*5
        deltaPosS = moteur.move_motor(0,thetaS)
        essais=1
        
        #en cas d'erreur, on recommence jusqu'à réussir à se positionner correctement
        while deltaPosS!=0 and essais<6:
            time.sleep(0.5)
            moteur.move_motor(0,thetaS+5)
            time.sleep(0.5)
            deltaPosS = moteur.move_motor(0,thetaS)
            essais = essais + 1
        if essais = 6:
            print("Erreur : Impossible de positionner la source en theta = ", thetaS)
            #faire quitter la fonction
        
        #A ce point du programme, on a donc correctement positionné la source.
        
        #On commence par s'occuper du rayon réfléchi
        
        #On déplace le capteur jusqu'à la bonne position
        thetaC = trouverPositionReflechi()
        if moteur.move_motor(1,thetaC)!=0:
            compteur = 1
        while compteur > 0 and compteur < 6:
            time.sleep(0.5)
            moteur.move_moteur(1,thetaC+5)
            time.sleep(0.5)
            if moteur.move_moteur(1,thetaC)!=0:
                compteur = compteur + 1
        if compteur = 6:
            print("Erreur : impossible de rejoindre la position correspondant du rayon réfléchi en theta = ", thetaS)
            #faire quitter la fonction
        
        #faire l'acquisition pour le rayon réfléchi, en faisant varier la polarisation
        
        #on s'occupe du rayon transmis
        
        #on déplace le capteur jusqu'à la bonne position
        thetaC = trouverPositionTransmis()
        compteur = 0
        if moteur.move_motor(1,thetaC)!=0:
            compteur = 1
        while compteur > 0 and compteur < 6:
            time.sleep(0.5)
            moteur.move_moteur(1,thetaC+5)
            time.sleep(0.5)
            if moteur.move_moteur(1,thetaC)!=0:
                compteur = compteur + 1
        if compteur = 6:
            print("Erreur : impossible de rejoindre la position correspondant du rayon transmis en theta = ", thetaS)
            #faire quitter la fonction
            
        #faire l'acquisition pour le rayon transmis, en faisant varier la polarisation
        
    return 0

SyntaxError: invalid syntax (<ipython-input-1-2c829d9bb21d>, line 27)

In [ ]:
#interface

from tkinter import *
import tkinter.font as tkFont
import matplotlib
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.figure import Figure
import seabreeze.spectrometers as sb
import webbrowser
import numpy as np
import motor as m
import core_sam2 as sam

#création du menu racine
racine = Tk()

ITRefl = IntVar() #Integration Time Reflexion
ITTrans = IntVar() #Integration Time Transmission

mot_accueil = Label(racine,text="Interface de gestion de SAM²",font='bold')
mot_accueil.pack()

saut0 = Label(racine, text="")
saut0.pack()

#acquisition manuelle
def acquisitionManuelle():
    panneauMesure = Toplevel(racine)
    titre = Label(panneauMesure, text='Acquisition manuelle')
    titre.pack()
    
    LAMBDAS=np.arange(400,900,5)
    X,Y=I_0(LAMBDAS)
    f = Figure(figsize=(12,6),dpi=100)
    a = f.add_subplot(111)
    a.plot(X,Y)
    canvas = FigureCanvasTkAgg(f, panneauMesure)
    canvas.show()
    canvas.get_tk_widget().pack()
    
    toolbar = NavigationToolbar2TkAgg(canvas,panneauMesure)
    toolbar.update()
    canvas._tkcanvas.pack()
    
    quitter1 = Button(panneauMesure,text='Quitter',command=panneauMesure.destroy)
    quitter1.pack()

#là il faut la fonction qui va bien, qui ressemble à celle de l'acquisition test dans calibrage

acquisition = Button(racine, text = "Réaliser une acquisition manuelle",command=acquisitionManuelle)
acquisition.pack()

#calibrage

#la fonction qui sera utilisée par le bouton
def calibrage():
    
    #On crée une nouvelle fenêtre pour le calibrage
    
    fenCalibrage = Toplevel(racine)
    titre0 = Label(fenCalibrage, text="Calibrage de l'instrument", font='bold')
    titre0.pack()
    saut2 = Label(fenCalibrage, text="")
    saut2.pack()
    def positionnerBras():
        moteur=m.Moteurs()
        moteur.move_motor(1,300,25)
        moteur.move_motor(0,208,25)
    initialisation = Button(fenCalibrage, text='Positionner les bras pour le calibrage',command=positionnerBras)
    initialisation.pack()
    
    
    
    #On va mettre toutes les instructions pour le calibrage
    
    saut4 = Label(fenCalibrage, text="")
    saut4.pack()
    cadreInstructions = LabelFrame(fenCalibrage, text="Calirage planéité", bg='white')
    cadreInstructions.pack()
    instructions1 = Label(cadreInstructions, text='- Après avoir allumé la source et le spectromètre, utilisez le bouton "Positionner les bras pour le calibrage" pour placer les bras face à face, sans échantillon.', bg = 'white')
    instructions1.pack()
    instructions2 = Label(cadreInstructions, text="- Si les deux bras ne sont pas parfaitement l'un en face de l'autre, déplacez précautionneusement à la main le bras du capteur de quelques millimètres pour ajuster le positionnement.", bg = 'white')
    instructions2.pack()
    instructions3 = Label(cadreInstructions, text="- Utilisez les vis des portes-fibre pour que le faisceau émis atteigne le centre de la lentille du capteur. Ce réglage assure la planéité du système.", bg = 'white')
    instructions3.pack()
    saut3 = Label(fenCalibrage, text="")
    saut3.pack()
    
    quitter0 = Button(fenCalibrage,text='Fermer',command=fenCalibrage.destroy)
    quitter0.pack()

calibrage = Button(racine, text = "Calibrer l'instrument",command=calibrage)
calibrage.pack()

#Mesure complète sur l'échantillon

#La fonction qui sera utilisée par le bouton

def Auto_tot():
    moteur=sam.m.Moteurs()
    LAMBDAS=np.arange(400,800,5)
    T=sam.Integration_time()
    print(T)
    LAMBDAS,I0=sam.I_0(LAMBDAS,T,10,25)
    #plt.plot(LAMBDAS,I0)
    
    #on va ouvrir un nouvelle fenêtre pour poser une question
    inter = Toplevel(racine)
    text1 = Label(inter, text="Le temps d'intégration idéal a maintenant été calculé.")
    text1.pack()
    text2 = Label(inter, text="Placez l'échantillon sur la plateforme, et assurez-vous de son bon positionnement, notamment vertical. Puis, cliquez sur 'OK'.")
    text2.pack()
    def mesure():
        mesureEnCours = Toplevel(inter)
        text3 = Label(mesureEnCours,text='La mesure est en cours ! Vous pouvez aller prendre un café.')
        text3.pack()
        I_trans=sam.Auto_transmission(moteur,T,LAMBDAS,I0)
        I_reflex=sam.Auto_reflexion(moteur,T,LAMBDAS,I0)
        finMesure = Toplevel(mesureEnCours)
        text4 = Label(finMesure,text="Ca y est, la mesure est terminée !")
        text4.pack()
        boutonFin = Button(finMesure, text='Tout fermer',command=inter.destroy)
        boutonFin.pack()
        return I_trans, I_reflex
    suite = Button(inter, text='OK',command=mesure)
    suite.pack()

mesure = Button(racine, text = "Effectuer une mesure complète sur l'échantillon",command=Auto_tot)
mesure.pack()

saut1 = Label(racine, text="")
saut1.pack()

#Informations techniques

def Theory():
    webbrowser.open_new(r'file://C:\Users\Utilisateur\Desktop\Mecatro\Analyse_de_donnees\mueller.pdf')

theory_button = Button(racine, text="Theory used for data analysis",command=Theory)
theory_button.pack()
     
saut2 = Label(racine, text="")
saut2.pack()
   
credits = Label(racine, text="Powered by mecatro")
credits.pack()

racine.mainloop()

10
100
1000
10000
669900
1016
1011
1006
Done
4.9853372434
1002
in checking ...
(5775.599659330061, 6356.765775186872)
the precedent position was the wrong one
(6356.765775186872, 4556.641047207185)
the precedent position was the right one
987
in checking ...
(6685.927942174246, 6517.805249988985)
the precedent position was the right one
977
in checking ...
(5125.714546900097, 7478.344173179177)
the precedent position was the wrong one
(7478.344173179177, 5419.149190419469)
the precedent position was the right one
962
in checking ...
(7353.426870274469, 6127.341799657181)
the precedent position was the right one
954
in checking ...
(4644.144672231462, 7021.57701110624)
the precedent position was the wrong one
(7021.57701110624, 6611.801926294721)
the precedent position was the right one
939
in checking ...
(5303.3828099615, 7811.710402424545)
the precedent position was the wrong one
(7811.710402424545, 5611.257471109435)
the precedent position was the right one
926
in checking ...
(5487

In [2]:
import motor as m
moteur=m.Moteurs()


In [ ]:
moteur.move_motor(0,300,25)
moteur.move_motor(1,300,25)